# 3webアプリ作成

目次：https://docs.google.com/document/d/1eDw9PK5Ft0vImu81e7oERB_q-dZvxbFe8bGetZBgEEg/edit

## 3.3 応用実装

### 3.3.2 応用アプリ実装

#### ①天気アプリ

>アプリの概要

<img src="pic/img039.png">  

https://tenki-step2-ver6-sonl83xj4z.streamlit.app/

>動作を構成してる部品の紹介（APIやライブラリ）

API
- 天気予報 API（livedoor 天気互換）
    - 目的：天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返す
    - 参考：https://weather.tsukumijima.net/

ライブラリ
- streamlit
    - 目的：天気を表示するためのフロントエンドを生成する
- requests
    - 目的：天気情報のAPIをhttpリクエストするために使う
- datetime
    - 目的：現在時刻の取得や時間のフォーマットを活用するために使う
- pandas
    - 目的：天気APIで得られたデータを扱いやすくするために使う

>ライブラリのインストール

使用するライブラリ一覧
- streamlit
    - pip install streamlit
- requests
    - pip install requests
- datetime
    - Pythonの標準ライブラリなので、インストールはすでにされています。
- pandas
    - pip install pandas

>ライブラリの公式ドキュメントの見方や使い方を説明

- 天気API
    - リクエストの仕方
        - リクエストパラメータの項目で、例を見る
    - 取得するデータ
        - JSON データサンプルで得られるデータ一覧を見る。どこに欲しいデータが記載されているか確認
    - 参考：https://weather.tsukumijima.net/


ライブラリの詳細説明

- streamlit
    - インストール方法はget startページで確認
    - 参考：https://docs.streamlit.io/library/get-started/installation
    - st.titleなどはコンポーネントページから確認出来る
    - 参考：https://streamlit.io/components
- requests
    - 楽天APIでやったrequests.get()です。
    - 参考：https://pypi.org/project/requests/
- datetime
    - datetime.now()で現味時刻
    - datetime.now().hourで現在時間のhourを取得できます。
    - 参考：https://docs.python.org/ja/3/library/datetime.html
- pandas
    - pd.DataFrame()で使います。
    - 参考：https://pandas.pydata.org/docs/

>それぞれの機能をjupyterで動く最小限を作る

以下がjuptyterにて動く最低限のアプリの流れです。

1. 作るアプリの全体のコードを把握
1. それぞれ機能を解説
    1. 天気API
    1. 現在時刻の取得

#### 1.全体のコード

使うアプリの全体像は下記のようになります。  
これはweb上では動きませんが、手元では動作するアプリです。

以下のコードの動作の流れ
1. 拡張機能であるライブラリをインポート
1. URLとcitycodeで、そのcitycodeの天気を取得するためのURLを作る
1. requestsで天気APIにアクセスして天気情報を取得
1. 取得した天気のデータを使って降水確率のデータを取得
    1. 現在時刻を使って、現在の降水確率を取得
    1. 今日、明日、明後日の３日分の降水確率を構造化データとして表示


In [5]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import requests # リクエストするための機能をインポート
from datetime import datetime # 現在時刻などの時間を扱う機能をインポート
import pandas as pd # データフレームを扱う機能をインポート

print("天気アプリ")
print("東京の天気を表示しています。") 
print("東京のcitycodeは130010です。") 

url = "https://weather.tsukumijima.net/api/forecast/city/" + "130010" # APIにリクエストするURLを作成　東京のcityコード

response = requests.get(url) # 作成したリクエスト用URLでアクセスして、responseに代入

weather_json = response.json() # responseにjson形式の天気のデータが返ってくるので、response.json()をweather_jsonに代入
now_hour = datetime.now().hour # 現在の天気情報取得のために、現在時刻の時間をnow_hourに代入

# 今日の天気はweather_json['forecasts'][0]['chanceOfRain']
# 明日の天気はweather_json['forecasts'][1]['chanceOfRain']
# 明後日の天気はweather_json['forecasts'][2]['chanceOfRain']
# にそれぞれ格納されている

# 天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返すため、場合分けする。
if 0 <= now_hour and now_hour < 6:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T00_06'] # 今日の0-6時の降水確率を取得し、weather_nowに代入
elif 6 <= now_hour and now_hour < 12:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T06_12'] # 今日の6-12時の降水確率を取得し、weather_nowに代入
elif 12 <= now_hour and now_hour < 18:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T12_18'] # 今日の12-18時の降水確率を取得し、weather_nowに代入
else:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T18_24'] # 今日の18-24時の降水確率を取得し、weather_nowに代入

# 現在時刻の降水確率をweather_now_textに代入
weather_now_text = "現在の降水確率 : " + weather_now
print(weather_now_text) # 現在時刻の降水確率を表示

# 今日、明日、明後日の降水確率をDadaFrameに代入
df1 = pd.DataFrame(weather_json['forecasts'][0]['chanceOfRain'],index=["今日"]) # index名を今日という文字列に設定
df2 = pd.DataFrame(weather_json['forecasts'][1]['chanceOfRain'],index=["明日"]) # index名を明日という文字列に設定
df3 = pd.DataFrame(weather_json['forecasts'][2]['chanceOfRain'],index=["明後日"]) # index名を明後日という文字列に設定

df = pd.concat([df1,df2,df3]) # 今日、明日、明後日の降水確率を結合して一覧にしてdfに代入
print(df) # 一覧にした降水確率を表示

天気アプリ
東京の天気を表示しています。
東京のcitycodeは130010です。
現在の降水確率 : 20%
    T00_06 T06_12 T12_18 T18_24
今日     --%    --%    20%    70%
明日     90%    60%    10%    10%
明後日    50%    50%    50%    50%


2.それぞれ機能を解説

<img src="pic/img045.png"> 

2.1天気APIの仕様

<img src="pic/img046.png">

天気APIは、URLの末尾で取得する天気を変更できます。

例：  
https://weather.tsukumijima.net/api/forecast/city/130010　で東京の天気を取得  
https://weather.tsukumijima.net/api/forecast/city/270000　で大阪の天気を取得  

なので、東京の天気の場合は、  

In [6]:
#htttps://weather.tsukumijima.net/api/forecast/city/ + citycode（東京の場合は130010）でリクエストすると、天気の情報が返ってくる
requests.get("https://weather.tsukumijima.net/api/forecast/city/130010")

<Response [200]>

で取得することができます！  
jsonというテータ形式で返ってくるので、.json()をつけてpythonが読み取れる状態にしておきます。

まとめると以下のようなコードになります！

In [7]:
#htttps://weather.tsukumijima.net/api/forecast/city/ + citycode（東京の場合は130010）でリクエストする
response = requests.get("https://weather.tsukumijima.net/api/forecast/city/130010")

#リクエストした結果をjson形式で取得
weather_json = response.json() 

#必要であればJSONClackでJSONを整形すると良い

これで 変数weather_jsonに様々な東京の天気のデータが格納されています。  
この講座ではその中から降水確率のデータを使用します。

<img src="pic/img047.png">

今日の天気は  
weather_json['forecasts'][0]  
に格納されています。

それぞれの天気のデータは配列として指定で取得できます。
- 今日　：weather_json['forecasts'][0] 
- 明日　：weather_json['forecasts'][1] 
- 明後日：weather_json['forecasts'][2] 

今日の天気のデータを確認してみましょう

In [8]:
# 今日の天気はweather_json['forecasts'][0]で表示できる
weather_json['forecasts'][0]

{'date': '2023-06-08',
 'dateLabel': '今日',
 'telop': '晴のち雨',
 'detail': {'weather': '晴れ\u3000後\u3000くもり\u3000夜\u3000雨',
  'wind': '南の風',
  'wave': '０．５メートル'},
 'temperature': {'min': {'celsius': None, 'fahrenheit': None},
  'max': {'celsius': '28', 'fahrenheit': '82.4'}},
 'chanceOfRain': {'T00_06': '--%',
  'T06_12': '--%',
  'T12_18': '20%',
  'T18_24': '70%'},
 'image': {'title': '晴のち雨',
  'url': 'https://www.jma.go.jp/bosai/forecast/img/112.svg',
  'width': 80,
  'height': 60}}

この講義では、降水確率を使うため、  
weather_json['forecasts'][今日明日明後日のどれか]['chanceOfRain'][ここに時間指定]  
でそれぞれの区間の降水確率を取得します。

例として、以下のコードになります。

In [9]:
# 色々な情報が入ってくるため、必要な情報をprintして挙動を確認してみる
# 今日の天気
print("--今日の降水確率--")
print("0-6時")
print(weather_json['forecasts'][0]['chanceOfRain']['T00_06'])
print("6-12時")
print(weather_json['forecasts'][0]['chanceOfRain']['T06_12'])
print("12-18時")
print(weather_json['forecasts'][0]['chanceOfRain']['T12_18'])
print("18-24時")
print(weather_json['forecasts'][0]['chanceOfRain']['T18_24'])

# 明日の天気
print("--明日の降水確率--")
print("0-6時")
print(weather_json['forecasts'][1]['chanceOfRain']['T00_06'])
print("6-12時")
print(weather_json['forecasts'][1]['chanceOfRain']['T06_12'])
print("12-18時")
print(weather_json['forecasts'][1]['chanceOfRain']['T12_18'])
print("18-24時")
print(weather_json['forecasts'][1]['chanceOfRain']['T18_24'])

# あさっての天気
print("--明後日の降水確率--")
print("0-6時")
print(weather_json['forecasts'][2]['chanceOfRain']['T00_06'])
print("6-12時")
print(weather_json['forecasts'][2]['chanceOfRain']['T06_12'])
print("12-18時")
print(weather_json['forecasts'][2]['chanceOfRain']['T12_18'])
print("18-24時")
print(weather_json['forecasts'][2]['chanceOfRain']['T18_24'])

--今日の降水確率--
0-6時
--%
6-12時
--%
12-18時
20%
18-24時
70%
--明日の降水確率--
0-6時
90%
6-12時
60%
12-18時
10%
18-24時
10%
--明後日の降水確率--
0-6時
50%
6-12時
50%
12-18時
50%
18-24時
50%


次は現在時刻を使って、現在の降水確率を取得します。

<img src="pic/img048.png">

現在時刻の取得は  
datetime.now()を使います。

今回は、現在時刻の時間が欲しいので、.hourで指定します。  
これによって数字で得られるので、0-6時、6-12時、12-18時、18-24時の条件文に使えます。

In [10]:
datetime.now().hour # 現在時刻の時間を取得

13

この得られた数字を使って、現在の降水確率を取得します。

In [11]:
# 天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返すため、場合分けする。
now_hour = datetime.now().hour  # 現在時刻の時間を取得し、これを条件にする

if 0 <= now_hour and now_hour < 6:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T00_06'] # 今日の0-6時の降水確率を取得し、weather_nowに代入
elif 6 <= now_hour and now_hour < 12:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T06_12'] # 今日の6-12時の降水確率を取得し、weather_nowに代入
elif 12 <= now_hour and now_hour < 18:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T12_18'] # 今日の12-18時の降水確率を取得し、weather_nowに代入
else:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T18_24'] # 今日の18-24時の降水確率を取得し、weather_nowに代入

# 現在時刻の降水確率をweather_now_textに代入
weather_now_text = "現在の降水確率 : " + weather_now
print(weather_now_text) # 現在時刻の降水確率を表示

現在の降水確率 : 20%


2.3今日、明日、明後日の３日分の降水確率を構造化データとして表示

<img src="pic/img049.png">

pandasのデータフレームにそれぞれ降水確率のデータを入れます。  
pd.DataFrame(ここにデータ)でデータフレーム化することができます。

わかりやすいようにindexは今日、明日、明後日と付けると以下のようになります。


In [12]:
# 今日、明日、明後日の降水確率をDadaFrameに代入
# df1, df2, df3としてみて、一つ一つ表示してみる
df1 = pd.DataFrame(weather_json['forecasts'][0]['chanceOfRain'],index=["今日"]) # index名を今日という文字列に設定
df2 = pd.DataFrame(weather_json['forecasts'][1]['chanceOfRain'],index=["明日"]) # index名を明日という文字列に設定
df3 = pd.DataFrame(weather_json['forecasts'][2]['chanceOfRain'],index=["明後日"]) # index名を明後日という文字列に設定

それぞれのデータフレームを確認すると

In [13]:
print("--今日の降水確率--")
display(df1)
print("--明日の降水確率--")
display(df2)
print("--明後日の降水確率--")
display(df3)

--今日の降水確率--


,T00_06,T06_12,T12_18,T18_24
今日,--%,--%,20%,70%


--明日の降水確率--


,T00_06,T06_12,T12_18,T18_24
明日,90%,60%,10%,10%


--明後日の降水確率--


,T00_06,T06_12,T12_18,T18_24
明後日,50%,50%,50%,50%


それぞれのデータフレームを１つにしたいので、  
pandasの機能である pd.concatで結合します。

使い方は pd.concat([ここにデータ,ここにデータ])でできます。  
結合すると以下のようになります。

In [14]:
df = pd.concat([df1,df2,df3]) # 今日、明日、明後日の降水確率を結合して一覧にしてdfに代入
df

,T00_06,T06_12,T12_18,T18_24
今日,--%,--%,20%,70%
明日,90%,60%,10%,10%
明後日,50%,50%,50%,50%


以上がアプリの動作部分です。

>streamlitに実装する場合の使う機能

今はユーザーがブラウザ上で選択したりなど、想定していませんが、  
選択肢の表示などのコンポーネントを紹介します。

※streamlitはjupyterでは動作せず、実行ファイルでのみ動作します

タイトルは  
st.title("ここにタイトル")

ただの表示は  
st.write("ここに表示したい文字")

ただし、再読み込みしない限り、この表示された文字は変更されません。

変数を監視して表示させる方法は以下のようになります。
1. 表示させる場所を確保する
1. その空間に.writeで書き込みをする

例として、変数testを使います。

In [ ]:
test = st.empty()
test.write("ここに書き込みたい事")

pandasのデータフレームを表示したい場合は、  
st.dataframe(ここにデータ)  
で表示できます。

citycodeに使う選択肢は  st.selectbox("ここに表示したい項目名",ここに配列のデータ)

でできます。

以上の機能を使ってwebアプリにしていきます。

>（演習）streamlitに実装しましょう！

手元で動いたアプリをstreamlitで動くようにします。  
この演習では必ず実行ファイルにコーディングしてください。

※streamlit run ファイル名で実行する時は必ず実行ファイルで実行してください。

実装の流れ
1. URLを選択できるように変更
    1. citycodeを変数化
    1. citycodeにセレクトボックスの値が入るようにする
1. 手元のコードに反映
1. ターミナルに出力していた文字などをブラウザに出力

以下の手元で動いたコードをstreamlitで動くようにします。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import requests # リクエストするための機能をインポート
from datetime import datetime # 現在時刻などの時間を扱う機能をインポート
import pandas as pd # データフレームを扱う機能をインポート

print("天気アプリ")

url = "https://weather.tsukumijima.net/api/forecast/city/" + "130010" # APIにリクエストするURLを作成　東京のcityコード

response = requests.get(url) # 作成したリクエスト用URLでアクセスして、responseに代入

weather_json = response.json() # responseにjson形式の天気のデータが返ってくるので、response.json()をweather_jsonに代入
now_hour = datetime.now().hour # 現在の天気情報取得のために、現在時刻の時間をnow_hourに代入

# 今日の天気はweather_json['forecasts'][0]['chanceOfRain']
# 明日の天気はweather_json['forecasts'][1]['chanceOfRain']
# 明後日の天気はweather_json['forecasts'][2]['chanceOfRain']
# にそれぞれ格納されている

# 天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返すため、場合分けする。
if 0 <= now_hour and now_hour < 6:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T00_06'] # 今日の0-6時の降水確率を取得し、weather_nowに代入
elif 6 <= now_hour and now_hour < 12:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T06_12'] # 今日の6-12時の降水確率を取得し、weather_nowに代入
elif 12 <= now_hour and now_hour < 18:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T12_18'] # 今日の12-18時の降水確率を取得し、weather_nowに代入
else:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T18_24'] # 今日の18-24時の降水確率を取得し、weather_nowに代入

# 現在時刻の降水確率をweather_now_textに代入
weather_now_text = "現在の降水確率 : " + weather_now
print(weather_now_text) # 現在時刻の降水確率を表示

# 今日、明日、明後日の降水確率をDadaFrameに代入
df1 = pd.DataFrame(weather_json['forecasts'][0]['chanceOfRain'],index=["今日"]) # index名を今日という文字列に設定
df2 = pd.DataFrame(weather_json['forecasts'][1]['chanceOfRain'],index=["明日"]) # index名を明日という文字列に設定
df3 = pd.DataFrame(weather_json['forecasts'][2]['chanceOfRain'],index=["明後日"]) # index名を明後日という文字列に設定

df = pd.concat([df1,df2,df3]) # 今日、明日、明後日の降水確率を結合して一覧にしてdfに代入
print(df) # 一覧にした降水確率を表示

1.URLを選択できるように変更

<img src="pic/img051.png">

1.1citycodeを変数化

変更の流れ
1. 変数を使う
1. 変数の値にセレクトボックスの値が入るようにする

以下のコードの部分をセレクトボックスで選択できるように変更します。

In [20]:
url = "https://weather.tsukumijima.net/api/forecast/city/" + "130010" 
print(url)

https://weather.tsukumijima.net/api/forecast/city/130010


URLを変数city_codeを使ってで変動できるようにします。

In [22]:
citycode = "130010" 

url = "https://weather.tsukumijima.net/api/forecast/city/" + citycode
print(url)

https://weather.tsukumijima.net/api/forecast/city/130010


citycodeに適応される文字列を辞書型配列を使って、人間に理解できる選択肢に変更します。  
この配列の変数名はcity_code_listとしておきます。

In [23]:
city_code_list = {
    "東京都":"130010",
}

citycode = city_code_list["東京都"] 

url = "https://weather.tsukumijima.net/api/forecast/city/" + citycode
print(url)

https://weather.tsukumijima.net/api/forecast/city/130010


1.2citycodeにセレクトボックスの値が入るようにする

streamlitのセレクトボックスを使って、東京の選択肢を作ります。  
使い方はst.selectbox("ここに表示したい項目名",ここに配列のデータ)なので、以下のコードになります。

セレクトボックスの値はcity_code_indexに代入します。  
辞書型の配列はcity_code_list.keys()で取得できます。

※ここからjupyterでは実行できません。

In [25]:
city_code_list.keys()

dict_keys(['東京都'])

In [ ]:
city_code_list = {
    "東京都":"130010",
}

city_code_index = st.selectbox("地域を選んでください。",city_code_list.keys())

citycode = city_code_list[city_code_index] 

url = "https://weather.tsukumijima.net/api/forecast/city/" + citycode
print(url)

2.手元のコードに反映

streamlitに実装したアプリをまとめると、以下のようになります。

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import requests # リクエストするための機能をインポート
from datetime import datetime # 現在時刻などの時間を扱う機能をインポート
import pandas as pd # データフレームを扱う機能をインポート




print("天気アプリ")


# 選択肢を作成
city_code_list = {
    "東京都":"130010",
}

city_code_index = st.selectbox("地域を選んでください。",city_code_list.keys()) # 選択肢のキーをst.selectboxで選択し、city_code_indexに代入
city_code = city_code_list[city_code_index] # 選択したキーからAPIのリクエストに使うcityコードに変換し、city_codeに代入

url = "https://weather.tsukumijima.net/api/forecast/city/" + city_code # APIにリクエストするURLを作成


response = requests.get(url) # 作成したリクエスト用URLでアクセスして、responseに代入

weather_json = response.json() # responseにjson形式の天気のデータが返ってくるので、response.json()をweather_jsonに代入
now_hour = datetime.now().hour # 現在の天気情報取得のために、現在時刻の時間をnow_hourに代入

# 今日の天気はweather_json['forecasts'][0]['chanceOfRain']
# 明日の天気はweather_json['forecasts'][1]['chanceOfRain']
# 明後日の天気はweather_json['forecasts'][2]['chanceOfRain']
# にそれぞれ格納されている

# 天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返すため、場合分けする。
if 0 <= now_hour and now_hour < 6:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T00_06'] # 今日の0-6時の降水確率を取得し、weather_nowに代入
elif 6 <= now_hour and now_hour < 12:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T06_12'] # 今日の6-12時の降水確率を取得し、weather_nowに代入
elif 12 <= now_hour and now_hour < 18:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T12_18'] # 今日の12-18時の降水確率を取得し、weather_nowに代入
else:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T18_24'] # 今日の18-24時の降水確率を取得し、weather_nowに代入

# 現在時刻の降水確率をweather_now_textに代入
weather_now_text = "現在の降水確率 : " + weather_now
print(weather_now_text) # 現在時刻の降水確率を表示

# 今日、明日、明後日の降水確率をDadaFrameに代入
df1 = pd.DataFrame(weather_json['forecasts'][0]['chanceOfRain'],index=["今日"]) # index名を今日という文字列に設定
df2 = pd.DataFrame(weather_json['forecasts'][1]['chanceOfRain'],index=["明日"]) # index名を明日という文字列に設定
df3 = pd.DataFrame(weather_json['forecasts'][2]['chanceOfRain'],index=["明後日"]) # index名を明後日という文字列に設定

df = pd.concat([df1,df2,df3]) # 今日、明日、明後日の降水確率を結合して一覧にしてdfに代入
print(df) # 一覧にした降水確率を表示

3.ターミナルに出力していた文字などをブラウザに出力

最後にprintでターミナルで出力していた  
タイトルや文字やデータフレームを以下の４つのコードを使ってブラウザに出力します。

- st.title
- st.write
- st.empty()
- st.dataframe

<img src="pic/img050.png">

In [ ]:
import streamlit as st # フロントエンドを扱うstreamlitの機能をインポート
import requests # リクエストするための機能をインポート
from datetime import datetime # 現在時刻などの時間を扱う機能をインポート
import pandas as pd # データフレームを扱う機能をインポート

# 選択肢を作成
city_code_list = {
    "東京都":"130010",
    "大阪" : "270000",
}
# 選択肢のデフォルトを設定
city_code_index = "東京都"


st.title("天気アプリ") # タイトル
st.write("調べたい地域を選んでください。") # サブタイトル
city_code_index = st.selectbox("地域を選んでください。",city_code_list.keys()) # 選択肢のキーをst.selectboxで選択し、city_code_indexに代入
city_code = city_code_list[city_code_index] # 選択したキーからAPIのリクエストに使うcityコードに変換し、city_codeに代入
current_city_code = st.empty() # 選択中の地域を補油時するための箱をcurrent_city_codeとして用意
current_city_code.write("選択中の地域:" + city_code_index) # 用意した箱に選択肢した地域を代入し、表示させる

url = "https://weather.tsukumijima.net/api/forecast/city/" + city_code # APIにリクエストするURLを作成


response = requests.get(url) # 作成したリクエスト用URLでアクセスして、responseに代入

weather_json = response.json() # responseにjson形式の天気のデータが返ってくるので、response.json()をweather_jsonに代入
now_hour = datetime.now().hour # 現在の天気情報取得のために、現在時刻の時間をnow_hourに代入

# 今日の天気はweather_json['forecasts'][0]['chanceOfRain']
# 明日の天気はweather_json['forecasts'][1]['chanceOfRain']
# 明後日の天気はweather_json['forecasts'][2]['chanceOfRain']
# にそれぞれ格納されている

# 天気の情報を0-6時、6-12時、12-18時、18-24時の4つに分けて降水確率を今日、明日、明後日の３日間の天気を返すため、場合分けする。
if 0 <= now_hour and now_hour < 6:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T00_06'] # 今日の0-6時の降水確率を取得し、weather_nowに代入
elif 6 <= now_hour and now_hour < 12:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T06_12'] # 今日の6-12時の降水確率を取得し、weather_nowに代入
elif 12 <= now_hour and now_hour < 18:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T12_18'] # 今日の12-18時の降水確率を取得し、weather_nowに代入
else:
    weather_now = weather_json['forecasts'][0]['chanceOfRain']['T18_24'] # 今日の18-24時の降水確率を取得し、weather_nowに代入

# 現在時刻の降水確率をweather_now_textに代入
weather_now_text = "現在の降水確率 : " + weather_now
st.write(weather_now_text) # 現在時刻の降水確率を表示

# 今日、明日、明後日の降水確率をDadaFrameに代入
df1 = pd.DataFrame(weather_json['forecasts'][0]['chanceOfRain'],index=["今日"]) # index名を今日という文字列に設定
df2 = pd.DataFrame(weather_json['forecasts'][1]['chanceOfRain'],index=["明日"]) # index名を明日という文字列に設定
df3 = pd.DataFrame(weather_json['forecasts'][2]['chanceOfRain'],index=["明後日"]) # index名を明後日という文字列に設定

df = pd.concat([df1,df2,df3]) # 今日、明日、明後日の降水確率を結合して一覧にしてdfに代入
st.dataframe(df) # 一覧にした降水確率を表示